# Memantau Model

Saat Anda menyebarkan model ke dalam produksi sebagai layanan, Anda akan ingin memantaunya untuk melacak penggunaan dan menjelajahi permintaan yang diprosesnya. Anda dapat menggunakan Azure Application Insights untuk memantau aktivitas titik akhir layanan model.

## Menghubungkan ke ruang kerja Anda

Untuk memulai, hubungkan ke ruang kerja Anda.

> **Catatan**: Jika Anda belum membuat sesi yang terautentikasi dengan langganan Azure, Anda akan diminta untuk mengautentikasi dengan mengklik tautan, memasukkan kode autentikasi, dan masuk ke Azure.

In [ ]:
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to work with', ws.name)

## Menyiapkan model untuk penyebaran

Sekarang kita membutuhkan model untuk disebarkan. Jalankan kode di bawah ini untuk:

1. Membuat dan mendaftarkan himpunan data.
2. Latih model menggunakan set data.
3. Daftarkan modelnya.

In [ ]:
from azureml.core import Experiment
from azureml.core import Model
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score, roc_curve
from azureml.core import Dataset

# Upload data files to the default datastore
default_ds = ws.get_default_datastore()
default_ds.upload_files(files=['./data/diabetes.csv', './data/diabetes2.csv'],
                       target_path='diabetes-data/',
                       overwrite=True,
                       show_progress=True)

#Create a tabular dataset from the path on the datastore
print('Creating dataset...')
data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, 'diabetes-data/*.csv'))

# Register the tabular dataset
print('Registering dataset...')
try:
    data_set = data_set.register(workspace=ws, 
                               name='diabetes dataset',
                               description='diabetes data',
                               tags = {'format':'CSV'},
                               create_new_version=True)
except Exception as ex:
    print(ex)

# Create an Azure ML experiment in your workspace
experiment = Experiment(workspace=ws, name='mslearn-train-diabetes')
run = experiment.start_logging()
print("Starting experiment:", experiment.name)

# load the diabetes dataset
print("Loading Data...")
diabetes = data_set.to_pandas_dataframe()

# Separate features and labels
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# Train a decision tree model
print('Training a decision tree model')
model = DecisionTreeClassifier().fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

# Save the trained model
model_file = 'diabetes_model.pkl'
joblib.dump(value=model, filename=model_file)
run.upload_file(name = 'outputs/' + model_file, path_or_stream = './' + model_file)

# Complete the run
run.complete()

# Register the model
print('Registering model...')
run.register_model(model_path='outputs/diabetes_model.pkl', model_name='diabetes_model',
                   tags={'Training context':'Inline Training'},
                   properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})

# Get the registered model
model = ws.models['diabetes_model']

print('Model trained and registered.')

## Menyebarkan model sebagai layanan web

Sekarang Anda siap untuk menyebarkan model terdaftar sebagai layanan web.

Pertama, buat folder untuk file konfigurasi penyebaran

In [ ]:
import os

# Create a folder for the deployment files
deployment_folder = './diabetes_service'
os.makedirs(deployment_folder, exist_ok=True)
print(deployment_folder, 'folder created.')

# Set path for scoring script
script_file = 'score_diabetes.py'
script_path = os.path.join(deployment_folder,script_file)

Sekarang Anda memerlukan skrip entri yang akan digunakan layanan untuk mencetak data baru.

In [ ]:
%%writefile $script_path
import json
import joblib
import numpy as np
import os

# Called when the service is loaded
def init():
    global model
    # Get the path to the deployed model file and load it
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'diabetes_model.pkl')
    model = joblib.load(model_path)

# Called when a request is received
def run(raw_data):
    # Get the input data as a numpy array
    data = json.loads(raw_data)['data']
    np_data = np.array(data)
    # Get a prediction from the model
    predictions = model.predict(np_data)
    
    # print the data and predictions (so they'll be logged!)
    log_text = 'Data:' + str(data) + ' - Predictions:' + str(predictions)
    print(log_text)
    
    # Get the corresponding classname for each prediction (0 or 1)
    classnames = ['not-diabetic', 'diabetic']
    predicted_classes = []
    for prediction in predictions:
        predicted_classes.append(classnames[prediction])
    # Return the predictions as JSON
    return json.dumps(predicted_classes)

Sekarang Anda dapat menyebarkan layanan (dalam hal ini, sebagai Azure Container Instance (ACI).

> **Catatan**: Ini bisa memakan waktu beberapa menit - tunggu hingga status ditampilkan sebagai **Sehat**.

In [ ]:
from azureml.core import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice, Webservice

# Configure the scoring environment
aci_service_env = Environment(name='aci_service-env')
python_packages = ['scikit-learn', 'azureml-defaults', 'azure-ml-api-sdk']
for package in python_packages:
    aci_service_env.python.conda_dependencies.add_pip_package(package)
inference_config = InferenceConfig(source_directory=deployment_folder,
                                   entry_script=script_file,
                                   environment=aci_service_env)

# Configure the web service container
deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

# Deploy the model as a service
print('Deploying model...')
service_name = "diabetes-service-app-insights"
aci_service = Model.deploy(ws, service_name, [model], inference_config, deployment_config, overwrite=True)
aci_service.wait_for_deployment(show_output = True)
print(aci_service.state)

## Mengaktifkan Application Insights

Selanjutnya, Anda perlu mengaktifkan Application Insights untuk layanan tersebut.

In [ ]:
# Enable AppInsights
aci_service.update(enable_app_insights=True)
print('AppInsights enabled!')

## Menggunakan layanan web

Dengan layanan yang disebarkan, sekarang Anda dapat menggunakannya dari aplikasi klien.

Pertama, tentukan URL tempat aplikasi ini harus mengirimkan permintaannya.

In [ ]:
endpoint = aci_service.scoring_uri
print(endpoint)

Sekarang setelah Anda mengetahui URI titik akhir, aplikasi dapat dengan mudah membuat permintaan HTTP, mengirim data pasien dalam format JSON (atau biner), dan menerima kembali kelas yang diprediksi.

> **Tips**: Jika terjadi kesalahan karena titik akhir layanan belum siap. Tunggu beberapa detik dan coba lagi!

In [ ]:
import requests
import json

# Create new data for inferencing
x_new = [[2,180,74,24,21,23.9091702,1.488172308,22],
         [0,148,58,11,179,39.19207553,0.160829008,45]]

# Convert the array to a serializable list in a JSON document
input_json = json.dumps({"data": x_new})

# Set the content type
headers = { 'Content-Type':'application/json' }

# Get the predictions
predictions = requests.post(endpoint, input_json, headers = headers)
print(predictions.status_code)
if predictions.status_code == 200:
    predicted_classes = json.loads(predictions.json())
    for i in range(len(x_new)):
        print ("Patient {}".format(x_new[i]), predicted_classes[i] )

Sekarang Anda dapat melihat data yang dicatat untuk titik akhir layanan:

1. Di [Azure portal](https://portal.azure.com), buka ruang kerja Machine Learning Anda.
2. Pada halaman **Overview**, klik link untuk sumber daya **Application Insights** terkait.
3. Pada panel Application Insights, klik **Log**. 

    > **Catatan**: Jika ini pertama kalinya Anda membuka analitik log, Anda mungkin perlu mengklik **Memulai** untuk membuka editor kueri. Jika tip yang menjelaskan cara menulis kueri ditampilkan, tutup.

4. Tempel kueri berikut ke editor kueri dan klik **Jalankan**
    ```
    jejak
    | di mana pesan == "STDOUT"
      dan customDimensions.["Nama Layanan"] == "diabetes-service-app-insights"
    |stempel waktu proyek, customDimensions.Content
    ```
5. Lihat hasilnya. Pada awalnya mungkin tidak ada, karena layanan web ACI dapat memakan waktu hingga lima menit untuk mengirim telemetri ke Application Insights. Tunggu beberapa menit dan jalankan kembali kueri hingga Anda melihat data dan prediksi yang dicatat.
6. Setelah Anda meninjau data yang dicatat, tutup halaman kueri Application Insights.

## Menghapus layanan

Ketika Anda tidak lagi membutuhkan layanan, Anda harus menghapusnya.

> **Catatan**: Jika layanan sedang digunakan, Anda mungkin tidak dapat langsung menghapusnya.

In [ ]:
try:
    aci_service.delete()
    print('Service deleted.')
except Exception as ex:
    print(ex.message)

Untuk informasi selengkapnya tentang menggunakan Application Insights untuk memantau layanan yang diterapkan, lihat [dokumentasi Azure Machine Learning](https://docs.microsoft.com/azure/machine-learning/how-to-enable-app-insights).